In [ ]:
!pip show openai_agents
#https://pypi.org/project/openai-agents/

Name: openai-agents
Version: 0.0.10
Summary: OpenAI Agents SDK
Home-page: https://github.com/openai/openai-agents-python
Author: 
Author-email: OpenAI <support@openai.com>
License-Expression: MIT
Location: /home/jalcocert/Desktop/Multi-Agents/oaiagent_venv/lib/python3.13/site-packages
Requires: griffe, mcp, openai, pydantic, requests, types-requests, typing-extensions
Required-by: 


In [5]:

import os
from dotenv import load_dotenv
from openai import OpenAI  # pip install openai>0.28

# Load environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    api_key=api_key,
)

In [6]:
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

result = Runner.run_sync(agent, "Write a haiku about recursion in programming.")
print(result.final_output)

RuntimeError: This event loop is already running

In [3]:
from openai_agents.agent import Agent
from openai_agents.runner import Runner
from openai_agents.tools.web_search import WebSearchTool
from openai_agents.function_tool import function_tool

@function_tool
def get_current_weather(location: str, unit: str = "celsius") -> str:
    """Gets the current weather for a given location."""
    # In a real application, you would call a weather API here
    if location.lower() == "warsaw":
        return f"The current temperature in Warsaw is 15 degrees {unit}."
    else:
        return f"Weather information for {location} is not available."

ModuleNotFoundError: No module named 'openai_agents'

In [ ]:
weather_agent = Agent(
    name="Weather Reporter",
    instructions="You are a helpful assistant that provides current weather information.",
    tools=[get_current_weather],
    model="gpt-4o" # Or another suitable OpenAI model
)

search_agent = Agent(
    name="Web Searcher",
    instructions="You can search the web to find information.",
    tools=[WebSearchTool()],
    model="gpt-4o"
)

triage_agent = Agent(
    name="Task Router",
    instructions="Route the user to the appropriate agent based on their request.",
    handoffs=[weather_agent, search_agent],
    model="gpt-4o"
)

In [7]:
from agents import Agent, Runner
import asyncio

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
)


async def main():
    result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
    print(result.final_output)
    # ¡Hola! Estoy bien, gracias por preguntar. ¿Y tú, cómo estás?


if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop